# `nepare` ASAP `polaris` Competition

This notebook demonstrates using Neural Pairwise Regression (via `nepare`) with the `polaris` benchmarking library to compete in the ASAP discovery competition.

## Requirements
Python 3.10+ (originally run on 3.12)
 - polaris-lib
 - pandas
 - rdkit
 - lightning
 - torch
 - fastprop
 - chemprop 2.1
 - ipywidgets

You will also need to run `pip install .` in the repository's root directory to install `nepare`.

## `polaris` Setup

After running `polaris login` on the command line, we can import everything (checking that the version is recent enough) and then download the benchmark data.

In [1]:
import polaris as po
import pandas as pd

In [2]:
from packaging.version import Version
assert Version(po.__version__) >= Version("0.11.6"), "test.as_dataframe does not work in earlier versions of Polaris, please upgrade"

In [3]:
%%capture
# competition = po.load_competition("asap-discovery/antiviral-potency-2025")
# or
competition = po.load_competition("asap-discovery/antiviral-admet-2025")

Choose the type of embedding to use - either a fixed descriptor-based embedding with mordred (great with limited data) or a learnable embedding with ChemProp (more general representation).

In [4]:
EMBEDDING = "mordred"
#
# or
# 
# EMBEDDING = "chemprop"

In [5]:
train, test = competition.get_train_test_split()
test_df: pd.DataFrame = test.as_dataframe()
train_df: pd.DataFrame = train.as_dataframe()

In [6]:
train_df

,CXSMILES,LogD,MLM,HLM,MDR1-MDCKII,KSOL
0,COC1=CC=CC(Cl)=C1NC(=O)N1CCC[C@H](C(N)=O)C1 |a...,0.3,NaN,NaN,2.0,NaN
1,O=C(NCC(F)F)[C@H](NC1=CC2=C(C=C1Br)CNC2)C1=CC(...,2.9,NaN,NaN,0.2,333.0
2,O=C(NCC(F)F)[C@H](NC1=CC=C2CNCC2=C1)C1=CC(Br)=...,0.4,NaN,NaN,0.5,NaN
3,NC(=O)[C@H]1CCCN(C(=O)CC2=CC=CC3=C2C=CO3)C1 |&...,1.0,NaN,NaN,8.5,376.0
4,CC1=CC(CC(=O)N2CCC[C@H](C(N)=O)C2)=CC=N1 |&1:11|,-0.3,NaN,NaN,0.9,375.0
...,...,...,...,...,...,...
429,CC(C)NC[C@H](O)COC1=CC=CC2=CC=CC=C12 |&1:5|,NaN,63.0,25.5,NaN,NaN
430,O=C(O)CC1=CC=CC=C1NC1=C(Cl)C=CC=C1Cl,NaN,386.0,216.0,NaN,NaN
431,NCC1=CC(Cl)=CC(C(=O)NC2=CC=C3CNCC3=C2)=C1,2.0,NaN,NaN,NaN,NaN
432,COC(=O)NC1=NC2=CC=C(C(=O)C3=CC=CC=C3)C=C2N1,2.9,NaN,NaN,NaN,NaN


## Train Per-Task Models

In [7]:
import torch
import lightning
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint
from chemprop.nn.agg import MeanAggregation
from chemprop.nn.message_passing import BondMessagePassing
from fastprop.data import standard_scale, inverse_standard_scale

from nepare.data import PairwiseAugmentedDataset, PairwiseAnchoredDataset, PairwiseInferenceDataset
from nepare.nn import LearnedEmbeddingNeuralPairwiseRegressor, NeuralPairwiseRegressor
from nepare.inference import predict

from utils.splitting import split
from utils.chemprop_wrappers import smiles2molgraphcache, collate_fn, ChemPropEmbedder
from utils.mordred_wrappers import smi2features
from utils.metrics import evaluate_predictions

In [8]:
predictions = {}
tasks = list(competition.target_cols)
dataset_kwargs = dict(batch_size=64)
for task_n, task_name in enumerate(tasks):
    task_df = train_df[["CXSMILES", task_name]].copy()
    task_df.dropna(inplace=True)
    task_df.reset_index(inplace=True)
    train_idxs, val_idxs = split(task_df["CXSMILES"], test_df["CXSMILES"])
    train_targets = torch.tensor(task_df[task_name].iloc[train_idxs].to_numpy(), dtype=torch.float32).reshape(-1, 1)  # 2d!
    train_targets, target_means, target_vars = standard_scale(train_targets)
    val_targets = torch.tensor(task_df[task_name].iloc[val_idxs].to_numpy(), dtype=torch.float32).reshape(-1, 1)  # 2d!
    val_targets = standard_scale(val_targets, target_means, target_vars)
    if EMBEDDING == "mordred":
        Model = NeuralPairwiseRegressor
        train_features, feature_means, feature_vars = smi2features(task_df["CXSMILES"][train_idxs])
        val_features, _, _ = smi2features(task_df["CXSMILES"][val_idxs], feature_means, feature_vars)
        test_features, _, _ = smi2features(test_df["CXSMILES"], feature_means, feature_vars)
        # setup the datasets
        train_dataset = PairwiseAugmentedDataset(train_features, train_targets, how='full')
        val_dataset = PairwiseAnchoredDataset(train_features, train_targets, val_features, val_targets, how='full')
        val_absolute_dataset = PairwiseInferenceDataset(train_features, train_targets, val_features, how='full')
        test_dataset = PairwiseInferenceDataset(train_features, train_targets, test_features, how='full')
        # build the model
        npr = Model(train_features.shape[1], 70, 3)
    elif EMBEDDING == "chemprop":
        Model = LearnedEmbeddingNeuralPairwiseRegressor
        # featurize the data
        train_mgc = smiles2molgraphcache(task_df["CXSMILES"][train_idxs])
        val_mgc = smiles2molgraphcache(task_df["CXSMILES"][val_idxs])
        test_mgc = smiles2molgraphcache(test_df["CXSMILES"])
        # setup the datasets
        dataset_kwargs["collate_fn"] = collate_fn
        train_dataset = PairwiseAugmentedDataset(train_mgc, train_targets, how='sut')
        val_dataset = PairwiseAnchoredDataset(train_mgc, train_targets, val_mgc, val_targets, how='half')
        val_absolute_dataset = PairwiseInferenceDataset(train_mgc, train_targets, val_mgc, how='half')
        test_dataset = PairwiseInferenceDataset(train_mgc, train_targets, test_mgc, how='half')
        # build the model
        mp = BondMessagePassing(d_h=400)
        agg = MeanAggregation()
        embedder = ChemPropEmbedder(mp, agg)
        npr = Model(embedder, 400, 200, 2, lr=5e-5)

    # classic lightning training, inference
    train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, **dataset_kwargs)
    val_loader = torch.utils.data.DataLoader(val_dataset, **dataset_kwargs)
    val_absolute_loader = torch.utils.data.DataLoader(val_absolute_dataset, **dataset_kwargs)
    predict_loader = torch.utils.data.DataLoader(test_dataset, **dataset_kwargs)
    early_stopping = EarlyStopping(monitor="validation/loss", patience=3)
    model_checkpoint = ModelCheckpoint(monitor="validation/loss")
    trainer = lightning.Trainer(max_epochs=50, log_every_n_steps=1, callbacks=[early_stopping, model_checkpoint])
    trainer.fit(npr, train_loader, val_loader)
    npr = Model.load_from_checkpoint(model_checkpoint.best_model_path)
    y_pred, y_stdev = predict(npr, val_absolute_loader)
    y_pred = inverse_standard_scale(y_pred, target_means, target_vars)
    y_true = inverse_standard_scale(val_targets, target_means, target_vars)

    # metric reporting, for our own information
    print(f"{task_name=}:\n - {"\n - ".join([f'{name}: {score:.4f}' for name, score in evaluate_predictions(y_true.flatten().numpy(), y_pred.flatten().numpy()).items()])}")

    # actual predictions
    y_pred, y_stdev = predict(npr, predict_loader)
    y_pred = inverse_standard_scale(y_pred, target_means, target_vars)
    predictions[task_name] = y_pred.flatten().tolist()

100%|██████████| 226/226 [00:04<00:00, 48.35it/s]
/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/mordred/_base/pandas_module.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  t[t.applymap(is_missing)] = value
100%|██████████| 126/126 [00:02<00:00, 43.12it/s]
/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/mordred/_base/pandas_module.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  t[t.applymap(is_missing)] = value
100%|██████████| 126/126 [00:03<00:00, 39.55it/s]
/home/jackson/neural-pairwise-regression/.venv/lib/python3.12/site-packages/mordred/_base/pandas_module.py:17: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  t[t.applymap(is_missing)] = value
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


task_name='KSOL':
 - Pearson r: 0.8204
 - MSE: 8553.5254
 - MAE: 65.0838


Predicting: |          | 0/? [00:00<?, ?it/s]

This last block is commented out because it will fail (unless you are me) - you can replace the inputs with your own if you are submitting this for yourself.

In [9]:
# competition.submit_predictions(
#     predictions=predictions,
#     prediction_name="nepare",
#     prediction_owner="jacksonburns",
#     report_url="https://github.com/JacksonBurns/neural-pairwise-regression/blob/main/meta",
#     github_url = "https://github.com/JacksonBurns/neural-pairwise-regression/blob/main/notebooks/polaris_asap/main.ipynb",
#     description = "Neural Pairwise Regression with ChemProp as a learnable embedding",
# )